In [8]:
import json
import pandas as pd
import numpy as np

import PyPDF2
import re

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [22]:
path_pdf2 = "F:/Environmental Baseline Data/Version 2/Data/PDFs/A63951%2D1 V3 %2D ESA_Sec_1%2D11 %2D A4D9H6.pdf"
#Notorious PDF - Chapter Keys = 3XGHIHIH

In [3]:
path_pdf3 = "F:/Environmental Baseline Data/Version 2/Data/PDFs/A63951%2D3 V4 %2D ESA_Appendix B1_CS 134 Bowmanville %2D A4D9H8.pdf"
# works

path_pdf4 = "F:/Environmental Baseline Data/Version 2/Data/PDFs/A72401%2D10 V3 02 ESA Section 1 to Section 5%2DPart 1 %2D A4T0Z0.pdf"
#was not able to use the function to read the contents of the second page of the PDF 

path_pdf5 = "F:/Environmental Baseline Data/Version 2/Data/PDFs/A77032%2D2 V21_New_Pipeline_1of30 %2D A5A8D4.pdf"
# File missing in the folder 

path_pdf6 = "F:/Environmental Baseline Data/Version 2/Data/PDFs/A77035%2D2 V22_Atmospheric_Environment_AB %2D A5A8H9.pdf"
#Works

path_pdf7 = "F:/Environmental Baseline Data/Version 2/Data/PDFs/A77038%2D9 V22_Quebec_Poisson_et_habitat_du_poisson_1de22 %2D A5A8X1.pdf"
# File missing in the folder 

path_pdf8 = "F:/Environmental Baseline Data/Version 2/Data/PDFs/A63951%2D3 V4 %2D ESA_Appendix B1_CS 134 Bowmanville %2D A4D9H8.pdf"
# File missing in the folder 

path_pdf9 = "F:/Environmental Baseline Data/Version 2/Data/PDFs/A77017%2D2 V16_PartA_AB_Contents %2D A5A7L4.pdf"


path_pdf10= "F:/Environmental Baseline Data/Version 2/Data/PDFs/A63951%2D34 V4 %2D ESA_Appendix J%2D4 %2D A4D9L2.pdf"

In [5]:
#path_Text = "F:/Environmental Baseline Data/Version 2/Data/HTML Images and Tables/A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6/table-of-contents.txt"
path_pdf1 = "F:/Environmental Baseline Data/Version 2/Data/PDFs/A77017%2D2 V16_PartA_AB_Contents %2D A5A7L4.pdf"
# Works

path_pdfs = [path_pdf10]
path_Texts = []

for path_pdf in path_pdfs:
    path_Text = path_pdf.replace(" .pdf", "")
    if ".pdf" in path_Text:
        path_Text = path_pdf.replace(".pdf", "")
    #print(path_Text)
    path_Text = path_Text.replace("/PDFs", "/HTML Images and Tables")
    path_Text = path_Text + "/table-of-contents.txt"
    path_Texts.append(path_Text)
    #print(path_Text)
    
    
for i in range(len(path_pdfs)):
    print("The path of the PDF is: {0}".format(path_pdfs[i]))
    print("The path of the TOC is: {0}".format(path_Texts[i]))


    
#Though the arrays would be used when we are dealing with bulk PDFs 
# in this code, for now, I wil be working the string values instead of the arrays
    


The path of the PDF is: F:/Environmental Baseline Data/Version 2/Data/PDFs/A63951%2D34 V4 %2D ESA_Appendix J%2D4 %2D A4D9L2.pdf
The path of the TOC is: F:/Environmental Baseline Data/Version 2/Data/HTML Images and Tables/A63951%2D34 V4 %2D ESA_Appendix J%2D4 %2D A4D9L2/table-of-contents.txt


# Extract Chapter Headings from the TOC text file

### Reading the JSON File as text and then extracting teh main chapter names

Ref -> https://stackabuse.com/reading-and-writing-json-to-a-file-in-python/

In [10]:
def head_clean(head):
    index = 0
    for i in range(len(head)):
        if (head[i] == '.') and (head[i+1] == '.') and (head[i+2] =='.'):
            index = i
            break
        
    head_clean = head[:i]
    return head_clean

In [13]:
with open(path_Text) as json_file:
    data = json.load(json_file)
    raw_TOC_text = data['raw_text'] #currently the array has a lot of strings which are not main headers 
    if len(raw_TOC_text) < 1:
        print("No TOC Found")
        
    chapter_keys = ["0 Before First Chapter"] 
    Flag_Appendix_Found = 0
    for head in raw_TOC_text:
        head = head.strip()
        head = re.sub(' +', ' ',head)
        if head[0].isdigit() and len(head) > 5: # assuming all the headings in the TOC are numbered
            head = head_clean(head) # removing soo many filler '.' in the string
            
            ##########################################
            # Here, we night need to set a threshold (On the number of dots) value depending on the PDF and its TOC 
            ##########################################
            
            threshold = 2
            if head.count('.') < threshold:
                head = head.strip()
                chapter_keys.append(head)
                
            # Here we check for Appendix 
        if "Appendix" in head or "APPENDIX" in head:
            chapter_keys.append("99 Appendix")
            Flag_Appendix_Found = 1
            break
                
    if Flag_Appendix_Found == 0:
        chapter_keys.append("99 Appendix")
        

chapter_keys

No TOC Found


['0 Before First Chapter', '99 Appendix']

In [2]:
with open(path_Text) as json_file:
    data = json.load(json_file)

NameError: name 'path_Text' is not defined

In [393]:
PDF_locations = []
PDF_names = []
Chapter_numbers = []
Chapter_texts = []
index = -1
for chapter_key in chapter_keys:
    PDF_locations.append(path_pdf)
    PDF_names.append(path_pdf.split('/')[-1])
    if ' ' in chapter_key:
        index = chapter_key.index(' ')
    Chapter_numbers.append(chapter_key[:index])
    Chapter_texts.append(chapter_key[index+1:])

In [394]:
ChaptersDF = pd.DataFrame({'PDF_names' : PDF_names, 
                           'Chapter_numbers' : Chapter_numbers, 
                           'Chapter_texts' : Chapter_texts, 
                           'Chapter_keys' : chapter_keys, 
                           'PDF_locations' : PDF_locations}, dtype = str)
ChaptersDF

,PDF_names,Chapter_numbers,Chapter_texts,Chapter_keys,PDF_locations
0,A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf,0,Before First Chapter,0 Before First Chapter,F:/Environmental Baseline Data/Version 2/Data/...
1,A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf,13,Environmental Impact Assessment and Mitigation...,13 Environmental Impact Assessment and Mitigat...,F:/Environmental Baseline Data/Version 2/Data/...
2,A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf,13.1,Introduction,13.1 Introduction,F:/Environmental Baseline Data/Version 2/Data/...
3,A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf,13.2,Project Description,13.2 Project Description,F:/Environmental Baseline Data/Version 2/Data/...
4,A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf,13.3,Assessment Methods,13.3 Assessment Methods,F:/Environmental Baseline Data/Version 2/Data/...
5,A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf,13.4,Air,13.4 Air,F:/Environmental Baseline Data/Version 2/Data/...
6,A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf,13.5,Terrain and Soils,13.5 Terrain and Soils,F:/Environmental Baseline Data/Version 2/Data/...
7,A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf,13.6,Vegetation,13.6 Vegetation,F:/Environmental Baseline Data/Version 2/Data/...
8,A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf,13.7,Wildlife and Wildlife Habitat,13.7 Wildlife and Wildlife Habitat,F:/Environmental Baseline Data/Version 2/Data/...
9,A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf,13.8,Fisheries and Aquatic Resources,13.8 Fisheries and Aquatic Resources,F:/Environmental Baseline Data/Version 2/Data/...


# Reading PDF File to find the Chapter Headers

Ref -> https://stackoverflow.com/questions/17098675/searching-text-in-a-pdf-using-python

In [395]:
pdf_paths = [path_pdf]
for pdf_path in pdf_paths:
    # open the pdf file
    object = PyPDF2.PdfFileReader(path_pdf)
       
    ############################################
    # Here we should call the functions to extract the Chapter Names from the PDFs
    # input: Path of the text file 
    # output: ChaptersDF, chapter_keys
    ############################################
    
    # Searching the extracted Chapter Keys in the PDF and finding their number 
    
    # get number of pages
    NumPages = object.getNumPages()    
    # chapter_key_index is set as 1 because we dont want to search for"0 Before First Chapter"
    # the array chapter_key_pages has first element 0 for the same reason
    chapter_key_index = 1
    chapter_key_page_Nos = [0]
    Flag_TOC_over = 0
    # extract text and do the search
    
    for i in range(0, NumPages):
        PageObj = object.getPage(i)
        print("this is page " + str(i)) 
        Text = PageObj.extractText()
        Text = Text.lower()
        Text = Text.replace("\n", "")
        # Text = Text
        # print (Text)
            
        #ResSearch = re.search(chapter_keys[chapter_key_index], Text)
        key = chapter_keys[chapter_key_index].lower()
        while (key in Text) and (Flag_TOC_over == 1):
            chapter_key_page_Nos.append(i)
            # print("YESSSS Found")
            # print(key)
            chapter_key_index = chapter_key_index +1
            key = chapter_keys[chapter_key_index].lower()
        
        if "table of contents" in Text:
            Flag_TOC_over = 1
            chapter_key_index = 1 #Reset the index to start 
            # print("TOC Found") 
            
print(Flag_TOC_over)
chapter_key_page_Nos

this is page 0
this is page 1
this is page 2
this is page 3
this is page 4
this is page 5
this is page 6
this is page 7
this is page 8
this is page 9
this is page 10
this is page 11
this is page 12
this is page 13
this is page 14
this is page 15
this is page 16
this is page 17
this is page 18
this is page 19
this is page 20
this is page 21
this is page 22
this is page 23
this is page 24
this is page 25
this is page 26
this is page 27
this is page 28
this is page 29
this is page 30
this is page 31
this is page 32
this is page 33
this is page 34
this is page 35
this is page 36
this is page 37
this is page 38
this is page 39
this is page 40
this is page 41
this is page 42
this is page 43
this is page 44
this is page 45
this is page 46
this is page 47
this is page 48
this is page 49
this is page 50
this is page 51
this is page 52
this is page 53
this is page 54
this is page 55
this is page 56
this is page 57
this is page 58
this is page 59
this is page 60
this is page 61
this is page 62
th

[0, 24, 24, 27, 36, 42, 58, 83]

# Rough Work 

In [396]:
i = 24
object = PyPDF2.PdfFileReader(path_pdf)
key = chapter_keys[1].lower()
print(key)

PageObj = object.getPage(i)
print("this is page " + str(i)) 
Text = PageObj.extractText() 
Text = Text.lower()
print(Text)


if key in Text:
    print("YESSSS Found")
else:
    print("NOOOOOOOOO")
    


13 environmental impact assessment and mitigation plan
this is page 24
 environmental 
impact 
assessmentand 
mitigation 
plan
 page 13-1
   encana ekwan pipeline inc. 
13 environmental impact assessment and mitigation plan 
13.1 introduction 13.1.1 project background 
encana ekwan pipeline inc. (encana ekwan) is
 proposing to construct the ekwan pipeline 
(the project) to carry sweet 
natural gas from encana™s sierra plant located in the 
sierra/kyklo gas field in northeastern brit
ish columbia to the existing nova gas 
transmission ltd. (ngtl) pipeline system 
west of rainbow lake, alberta (see 
figure 13.1-1). as part of its f
iling requirements to the nationa
l energy board (neb) for an 
application for the pipeline, made
 pursuant to section 52 of the 
national energy board act
, encana ekwan has retained axys environmental
 consulting ltd. (
axys) to prepare this 
environmental impact assessment (eia) document 
to identify key bioph
ysical resources in 
the project area, the nat


In [397]:
Flag_TOC_over

1

In [398]:
"Table of Contents".lower()

'table of contents'

In [399]:
i = 24
Text = "13 environmental impact assessment and mitigation plan"
while ("13 Environmental Impact Assessment and Mitigation Plan".lower() in Text) and (i <10):
    print("YESSSS Found")
    
    i = i+1

In [400]:
i = 1

PageObj = object.getPage(i)
print("this is page " + str(i)) 
Text = PageObj.extractText() 
Text = Text.lower()
    
print(Text)

while ("13 Environmental Impact Assessment and Mitigation Plan".lower() in Text) and (i >1):
    print("YESSSS Found")
    
    i = i+1


this is page 1
 environmental 
impact 
assessmentand 
mitigation 
plan
page 13.ii   encana ekwan pipeline inc. 
table of contents 13 environmental impact assessment and mitigation plan............................................................13-1
 13.1 introduction..............................................................................................................
............13-1 
13.1.1 project background...............................................................................................13-1 

13.1.2 regulatory setting................................................................................................13-1 

13.1.3 scope of environmental assessment....................................................................13-3 
13.2 project description.......................................................................................................
........13-4 
13.2.1 scope of project......................................................................

In [186]:
"13  Environmental Impact Assessment and Mitigation Plan".lower()

'13  environmental impact assessment and mitigation plan'

In [187]:
Text

' environmental \nimpact \nassessmentand \nmitigation \nplan\n page 13-1\n   encana ekwan pipeline inc. \n13 environmental impact assessment and mitigation plan \n13.1 introduction 13.1.1 project background \nencana ekwan pipeline inc. (encana ekwan) is\n proposing to construct the ekwan pipeline \n(the project) to carry sweet \nnatural gas from encana™s sierra plant located in the \nsierra/kyklo gas field in northeastern brit\nish columbia to the existing nova gas \ntransmission ltd. (ngtl) pipeline system \nwest of rainbow lake, alberta (see \nfigure 13.1-1). as part of its f\niling requirements to the nationa\nl energy board (neb) for an \napplication for the pipeline, made\n pursuant to section 52 of the \nnational energy board act\n, encana ekwan has retained axys environmental\n consulting ltd. (\naxys) to prepare this \nenvironmental impact assessment (eia) document \nto identify key bioph\nysical resources in \nthe project area, the nat\nure of anticipated project effects on th

In [ ]:
sub = "gh"
MString = "Vibudh Singh"

ResSearch = re.search(sub, MString)
ResSearch

In [128]:
for i in range(1,5):
    print(i) 
    if i == 2:
        print(True)
        i = i-1

1
2
True
3
4


In [53]:
len("Figure 13.4-4  Mean Monthly Precipitation and Number of Days with Measurable\n")
len("Figure 13.7-2  Woodland Caribou Habitat Availability during the Early Winter Season..... 13-113\n")
len("Figure 13D.A-8  Typical Existing Bridge ...........................................................................................9\n",)

77

96

133

In [96]:
chapter = chapter_keys[2]
index = chapter.index(".")
index

2

In [67]:
d = {'col1': [1, 2], 'col2': [3, 4]}
df = pd.DataFrame(data=d)
df

,col1,col2
0,1,3
1,2,4


In [69]:
df2 = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
                    columns=['a', 'b', 'c'])
df2

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9
